<a href="https://colab.research.google.com/github/yoooobin/ESAA/blob/main/5_23_%EC%84%B8%EC%85%98_%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CIFAR-10 데이터셋을 이용한 CNN 모델

## Step 1. 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [14]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2. CIFAR-10 데이터셋 다운로드

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화 추가
])

# 데이터셋 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


100%|██████████| 170M/170M [00:03<00:00, 48.3MB/s]


##Step 3. DataLoader 설정

In [21]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## Step 4. CNN 모델 정의

In [22]:
# CNN 모델 정의

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()

        # 첫 번째 합성곱 층
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화 채널 수 입력

        # 두 번째 합성곱 층
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)

        # 세 번째 합성곱 층
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)

        # 풀링 층 (MaxPooling)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc1 = nn.Linear(128*2*2, 512)  # CIFAR-10은 32x32 크기
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # 활성화 함수 적용
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)  # Flatten

        x = F.relu(self.fc1(x))  # Fully Connected 활성화 함수 추가
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Step 5. 모델 학습 설정

In [23]:
# 모델을 초기화하세요.
model = CIFAR10_CNN();
model.to(device)

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Step 6. 모델 학습 루프

In [24]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분
num_epochs = 3
for epoch in range(num_epochs):
   for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값을 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()

   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/3], Loss: 1.1445
Epoch [2/3], Loss: 0.5780
Epoch [3/3], Loss: 0.7592


## Step 7. 테스트 정확도 평가

In [25]:
# 모델의 성능 평가

model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 72.61%


# 2. MNIST 데이터셋 분류 모델




## Step 1. 필요한 라이브러리 임포트

In [2]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [3]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2. MNIST 데이터셋 다운로드

In [4]:
# MNIST 데이터셋을 다운로드하고, 텐서로 변환할 수 있도록 필요한 전처리 함수를 추가하세요.
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 15.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.41MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.28MB/s]


##Step 3. DataLoader 설정

In [5]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## Step 4. CNN 모델 정의

In [6]:
# CNN 모델 정의
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()

        # 첫 번째 합성곱 블록
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # [1,28,28] -> [32,28,28]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # [32,28,28] -> [32,14,14]
        )

        # 두 번째 합성곱 블록
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),  # [32,14,14] -> [64,12,12]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  # [64,12,12] -> [64,6,6]
        )

        # 완전연결층
        self.fc1 = nn.Linear(in_features=64 * 6 * 6, out_features=600)
        self.drop = nn.Dropout(0.25)  # Dropout2d 대신 Dropout 사용
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)  # 10개 클래스

    def forward(self, x):
        out = self.layer1(x)             # [batch, 32, 14, 14]
        out = self.layer2(out)           # [batch, 64, 6, 6]
        out = out.view(out.size(0), -1)  # Flatten [batch, 2304]
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

## Step 5. 모델 학습 설정

In [7]:
# 모델을 초기화하세요.
model = MNISTCNN();
model.to(device)

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Step 6. 모델 학습 루프

In [8]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분

num_epochs = 3  # 에폭 수 3
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        train = Variable(images)
        labels = Variable(labels)

        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if not (count % 50):
            total = 0
            correct = 0
            with torch.no_grad():  # 평가 시 no_grad() 사용
                for images, labels in test_loader:
                    images, labels = images.to(device), labels.to(device)
                    test = Variable(images)
                    outputs = model(test)
                    predictions = torch.max(outputs, 1)[1]
                    correct += (predictions == labels).sum().item()
                    total += labels.size(0)

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

        if not (count % 500):
            print("Iteration: {}, Loss: {:.4f}, Accuracy: {:.2f}%".format(
                count, loss.item(), accuracy))

Iteration: 500, Loss: 0.0203, Accuracy: 97.21%
Iteration: 1000, Loss: 0.2322, Accuracy: 97.45%
Iteration: 1500, Loss: 0.0337, Accuracy: 98.46%
Iteration: 2000, Loss: 0.0231, Accuracy: 98.71%
Iteration: 2500, Loss: 0.0044, Accuracy: 98.51%


## Step 7. 테스트 정확도 평가

In [13]:
# 모델의 성능 평가

model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.42%
